<a href="https://colab.research.google.com/github/hbechara/HertieML/blob/main/Lesson6TrainingModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implement Batch Gradient Descent with early stopping (optional) for Softmax Regression (without using Scikit Learn).

# Setup

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "training_linear_models"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

np.random.seed(2042)

# Dataset

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
list(iris.keys())

['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename']

# What are the steps we will need to take to implement this model?

## Problem 1: Data

 - Split the Training set. Either use scikit learn's built-in split function or write your own.

In [ ]:
# Thilos Solution
# First: Extract the relevant data
# From the description Petal length and width seem to be highly correlated so let's extract these as the features
X = iris["data"][:,(2,3)]
y = iris["target"]
y.shape
# Ok we have 150 instances, so it is a really small data set.

# Let's split the data set.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 42)

print(len(y_train), y_train.shape)

# OneHotEncoder (manually)
# 1st way: Simple if/for loop

def y_converter(input_vector):
  y_converted = []
  for i in range(len(input_vector)):
    if input_vector[i] == 0:
      y_converted.append([1, 0, 0])
    elif input_vector[i] == 1:
      y_converted.append([0, 1, 0])
    elif input_vector[i] == 2:
      y_converted.append([0, 0, 1])
    else:
      print("Error! Category out of anticipated range...")
  return y_converted

# Call the function
y_train_conv = y_converter(y_train)
# Print
print(y_train, y_train_conv)

90 (90,)
[2 0 0 0 1 0 1 2 0 1 2 0 2 2 1 1 2 1 0 1 2 0 0 1 1 0 2 0 0 1 1 2 1 2 2 1 0
 0 2 2 0 0 0 1 2 0 2 2 0 1 1 2 1 2 0 2 1 2 1 1 1 0 1 1 0 1 2 2 0 1 2 2 0 2
 0 1 2 2 1 2 1 1 2 2 0 1 2 0 1 2] [[0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0,

- Remember: Softmax outputs a vector that represents the probability distributions of a list of potential outcomes.

- So write a small function to convert the vector of class indices into a matrix containing a one-hot vector for each instance. 

0 -> [1,0,0]

1 -> [0,1,0]

2 -> [0,0,1]

Book Solution for data splitting and encoding.
Note: this includes an additional function for bias, which we did not have time to cover in class. 

In [ ]:
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = iris["target"]

X_with_bias = np.c_[np.ones([len(X), 1]), X]

test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

In [ ]:
def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

In [ ]:
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

## Problem 2: Softmax

Tip 1: 
This is the equation that defines softmax:

$\sigma\left(\mathbf{s}(\mathbf{x})\right)_k = \dfrac{\exp\left(s_k(\mathbf{x})\right)}{\sum\limits_{j=1}^{K}{\exp\left(s_j(\mathbf{x})\right)}}$

Tip 2: You can find the code somewhere on stack overflow and adapt it.

In [ ]:
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

## Problem 3: Training.

- To train you need to model the loss and the gradient. 





The cost function:

$J(\mathbf{\Theta}) =
- \dfrac{1}{m}\sum\limits_{i=1}^{m}\sum\limits_{k=1}^{K}{y_k^{(i)}\log\left(\hat{p}_k^{(i)}\right)}$

The gradients:

$\nabla_{\mathbf{\theta}^{(k)}} \, J(\mathbf{\Theta}) = \dfrac{1}{m} \sum\limits_{i=1}^{m}{ \left ( \hat{p}^{(i)}_k - y_k^{(i)} \right ) \mathbf{x}^{(i)}}$

I have provided the code for you here, so you don't need to write the training loop if you implemented the softmax function correctly.


All I want you to do now is to add early stopping.

In [ ]:
# because we're using softmax and therefore a one-hot vector, we need to 
# make sure we randomly initialise Theta with the correct dimensions

n_inputs = X_train.shape[1] # == 3 (2 features plus the bias term)
n_outputs = len(np.unique(y_train))   # == 3 (3 iris classes)

Theta = np.random.randn(n_inputs, n_outputs)

In [ ]:
def loss_function(Y_train_one_hot):
  return -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))


In [ ]:
# And here is the code for batch gradient descent with softmax

eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

# we start by randomising theta 
Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    # converts the list to a probability list using softmax 
    Y_proba = softmax(logits) 
    if iteration % 500 == 0:
        # shows the loss every 500 iterations
        loss = loss_function(Y_train_one_hot)
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/len(X_train) * X_train.T.dot(error)
    # Update Theta
    Theta = Theta - eta * gradients

0 5.173284880908112
500 0.8258143504756522
1000 0.6740383508681776
1500 0.5891518016822946
2000 0.5353052890403674
2500 0.4975988211901051
3000 0.469220320068328
3500 0.4467104744290491
4000 0.4281482798645294
4500 0.41238656131534807
5000 0.3986986115898958


In [ ]:
# Thilos solution
# And here is the code for batch gradient descent with softmax

eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

# we start by randomising theta 
Theta = np.random.randn(n_inputs, n_outputs)

#Defining the loss function
def loss_function(Y_train_one_hot):
  return -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))

# Defining smallest loss
smallest_loss = np.infty


for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    # converts the list to a probability list using softmax 
    Y_proba = softmax(logits)
    # loss = loss_function(Y_train_one_hot) Not calculate it on every step, as it is costly.
    if iteration % 500 == 0:
        # shows the loss every 500 iterations
        loss = loss_function(Y_train_one_hot)
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/len(X_train) * X_train.T.dot(error)
    # Update Theta
    Theta = Theta - eta * gradients
    # Early stop 1
    loss_diff = smallest_loss - loss
    if loss_diff < epsilon:
      print("Loss improvement is super small! --> Entering early stopping")
      print("Loss improvement in this iteration: ", loss_diff)
      print("Threshold was epsilon: ", epsilon)
      break
    # Early stop 2
    if loss < smallest_loss:
      smallest_loss = loss
    else:
      print("Loss is getting bigger again! --> Entering early stopping")
      print("Loss of iteration before: ", iteration-1, smallest_loss)
      print("Loss of current iteration: ", iteration, loss)
      break
    # Early stop 1

    if loss < epsilon:
      print("Loss improvement is super small! --> Entering early stopping")



## Problem 4: Testing

- Ok we're almost done, now let's try some predictions:

- Now let's measure the accuracy on the test set.

In case we don't manage to go through it all, I've put the solutions in another colab [file](https://colab.research.google.com/drive/1_tblkukSgl50g9Od9dQ-ugNh9WTppjLE?usp=sharing): 